In [ ]:
# Folder with your Excel files
input_folder = "/Users/kevineriksson/Desktop/AnalysisFiles"
output_folder = "/Users/kevineriksson/Desktop/AnalysisFiles_Monthly"
files = [f for f in os.listdir(input_folder) if f.endswith(".xlsx")]

for file in files:
    file_path = os.path.join(input_folder, file)

    zone = os.path.splitext(file)[0].split('_')[-1].upper()

    print(f"Processing file: {file} with zone: {zone}")

    data = pd.read_excel(file_path)

    data['Hour'] = pd.to_datetime(data['Hour'], format="%d.%m.%Y %H:%M")
    data.set_index('Hour', inplace=True)

    spot_col = f"SPOT{zone}"
    solar_col = f"PRO{zone}SOL_ENTSOE"
    wind_col = f"PRO{zone}WINDON_ENTSOE"
    cnp_col = f"CNP{zone}"
    data[cnp_col] = data[cnp_col].astype(float) * 1000

    data['Solar Weighted'] = data[solar_col] * data[spot_col]

    data['Wind Weighted'] = data[wind_col] * data[spot_col]
    data['Solar Overgeneration'] = np.maximum(0, data[solar_col] - data[cnp_col])

    print(data['Solar Overgeneration'].head(20))

    monthly_data = data.resample('ME').apply({
        'Solar Weighted': 'sum',
        'Wind Weighted': 'sum',
        'Solar Overgeneration': 'sum',
        solar_col: 'sum',
        wind_col: 'sum',
        spot_col: 'mean',
        cnp_col: 'sum',
    })

    monthly_data['Capture Price Solar'] = monthly_data['Solar Weighted'] / monthly_data[solar_col]
    monthly_data['Capture Price Wind'] = monthly_data['Wind Weighted'] / monthly_data[wind_col]

    monthly_data['Capture Rate Wind'] = monthly_data['Capture Price Wind'] / monthly_data[f'SPOT{zone}']
    monthly_data['Capture Rate Solar'] = monthly_data['Capture Price Solar'] / monthly_data[f'SPOT{zone}']

    monthly_data['Solar Overgeneration'] = np.maximum(0, monthly_data['Solar Overgeneration'])
    monthly_data['Solar Overgeneration Share (%)'] = (monthly_data['Solar Overgeneration'] /
                                                      monthly_data[solar_col])

    monthly_data['Solar Cannibalization €'] = monthly_data['Capture Price Solar'] - monthly_data[f'SPOT{zone}']
    monthly_data['Wind Cannibalization €'] = monthly_data['Capture Price Wind'] - monthly_data[f'SPOT{zone}']

    monthly_data.index = monthly_data.index.date

    output_filename = f"MonthlyAnalysisSheet{zone}.xlsx"
    output_path = os.path.join(output_folder, output_filename)
    monthly_data.to_excel(output_path)